In [1]:
import pandas as pd

In [2]:
games = pd.read_csv('games.csv')

In [3]:
games.sample(2)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
13331,wYvU04xT,True,1.491650e+12,1.491651e+12,44,mate,black,45+30,arturfritz,1517,paspalov,1661,e4 c5 c4 g6 Nc3 a6 Nf3 Bg7 d3 Nc6 Be3 b6 Qc2 B...,B20,Sicilian Defense: Staunton-Cochrane Variation,3
13418,w89BRuTS,False,1.503219e+12,1.503221e+12,55,resign,black,20+20,zher0,1709,shaeila89,1384,e4 b6 d4 Bb7 Bd3 e6 Nf3 Nc6 O-O Nb4 c3 Nxd3 Qx...,B00,Owen Defense,2


In [4]:
def remove_variation(x):
    if ':' in x:
        return x[0:x.index(':')]
    return x

In [5]:
games["w_rating"] = games.white_rating.apply(lambda x: [x])
games["b_rating"] = games.black_rating.apply(lambda x: [x])
games["winner_white"] = games.winner.apply(lambda x: 1 if x == 'white' else 0)
games["winner_black"] = games.winner.apply(lambda x: 1 if x == 'black' else 0)
games["draw"] = games.winner.apply(lambda x: 1 if x == 'draw' else 0)
games["opening_moves"] = games.apply(lambda x: x.moves.split(" ")[0: x.opening_ply], axis = 1)
games["nb_games"] = 1
games["opening"] = games.opening_name.apply(lambda x: remove_variation(x))

In [6]:
games.sample(2)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,...,opening_name,opening_ply,w_rating,b_rating,winner_white,winner_black,draw,opening_moves,nb_games,opening
2518,3Z2zjtQb,True,1.490150e+12,1.490150e+12,76,mate,black,10+0,scorpion2248,1355,...,Scandinavian Defense,2,[1355],[1428],0,1,0,"[e4, d5]",1,Scandinavian Defense
12761,NTkSmCQ8,True,1.503648e+12,1.503648e+12,17,resign,white,30+0,victress,1689,...,Italian Game: Schilling-Kostic Gambit,6,[1689],[1677],1,0,0,"[e4, e5, Nf3, Nc6, Bc4, Nd4]",1,Italian Game


In [7]:
openings = games.groupby('opening').agg({'w_rating': 'sum', 'b_rating': 'sum', 
                                   'winner_white' : 'sum', 'winner_black' : 'sum', 'draw' : 'sum', 
                                              'opening_moves': 'min', 'nb_games': 'sum'})

In [8]:
popular_openings = openings.sort_values(by = 'nb_games', ascending = False).head(40)

In [9]:
popular_openings['rating_histogram'] = popular_openings.w_rating + popular_openings.b_rating
popular_openings.winner_white = popular_openings.apply(lambda x: int(x.winner_white * 100 / x.nb_games), axis = 1)
popular_openings.winner_black = popular_openings.apply(lambda x: int(x.winner_black * 100 / x.nb_games), axis = 1)
popular_openings.winner_draw = 100 - popular_openings.winner_white - popular_openings.winner_black

<ipython-input-9-30ae17fe31f6>:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  popular_openings.winner_draw = 100 - popular_openings.winner_white - popular_openings.winner_black


In [10]:
popular_openings.sample(1)

,w_rating,b_rating,winner_white,winner_black,draw,opening_moves,nb_games,rating_histogram
opening,,,,,,,,
Sicilian Defense,"[1250, 1500, 1113, 1358, 1200, 1203, 1422, 150...","[1002, 1094, 1423, 1036, 1002, 1019, 975, 978,...",45,49,127,"[Nc3, c5, e4, Nc6]",2573,"[1250, 1500, 1113, 1358, 1200, 1203, 1422, 150..."


In [11]:
def sorted_list(x):
    x.sort()
    return x

popular_openings.rating_histogram = popular_openings.rating_histogram.apply(lambda x: sorted_list(x))

In [28]:
popular_openings[['winner_white', 'winner_black', 'draw', 'opening_moves', 'nb_games', 'rating_histogram']].to_json('openings.json', orient = 'index')

In [12]:
popular_openings.to_pickle('openings.pkl')

## Computing the game flows 

In [30]:
games

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,...,opening_name,opening_ply,w_rating,b_rating,winner_white,winner_black,draw,opening_moves,nb_games,opening
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,...,Slav Defense: Exchange Variation,5,[1500],[1191],1,0,0,"[d4, d5, c4, c6, cxd5]",1,Slav Defense
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,...,Nimzowitsch Defense: Kennedy Variation,4,[1322],[1261],0,1,0,"[d4, Nc6, e4, e5]",1,Nimzowitsch Defense
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,...,King's Pawn Game: Leonardis Variation,3,[1496],[1500],1,0,0,"[e4, e5, d3]",1,King's Pawn Game
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,...,Queen's Pawn Game: Zukertort Variation,3,[1439],[1454],1,0,0,"[d4, d5, Nf3]",1,Queen's Pawn Game
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,...,Philidor Defense,5,[1523],[1469],1,0,0,"[e4, e5, Nf3, d6, d4]",1,Philidor Defense
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,...,Dutch Defense,2,[1691],[1220],1,0,0,"[d4, f5]",1,Dutch Defense
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,...,Queen's Pawn,2,[1233],[1196],0,1,0,"[d4, d6]",1,Queen's Pawn
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,...,Queen's Pawn Game: Mason Attack,3,[1219],[1286],1,0,0,"[d4, d5, Bf4]",1,Queen's Pawn Game
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,...,Pirc Defense,4,[1360],[1227],1,0,0,"[e4, d6, d4, Nf6]",1,Pirc Defense


In [126]:
g = 17
games.iloc[g].moves

'e4 e5 Nf3 Nc6 Bc4 Nf6 Nc3 Bc5 O-O O-O Ne1 Ne8 Qf3 Qg5 d4 Qg6 dxc5 Nd4 Qd1 d6 g3 Bh3 Nf3 Bxf1 Qxf1 Nxf3+ Kg2 Nh4+ Kh1 Nf3 Nd5 Qe6 Nxc7 Qc8 Nxa8 Qxa8 cxd6 Nxd6 Be2 Nd4 Bd3 Qc8 c3 Ne6 Qe1 Nc4 Bxc4 Qxc4 b3 Qc6 Ba3 Rd8 Be7 Rd7 Ba3 Ng5 Qd2 Rxd2 Kg2 Qxe4+ Kf1 Qe2+ Kg2 Qxf2+ Kh1 Qxh2#'

In [116]:
test = games.iloc[g].moves.split(" ")
initial_chessboard = {
     'wr-L': 'a1', 'wn-L': 'b1', 'wb-L': 'c1', 'wq':   'd1', 'wk':   'e1', 'wb-R': 'f1', 'wn-R': 'g1', 'wr-R': 'h1',
      'wp-a': 'a2', 'wp-b': 'b2', 'wp-c': 'c2', 'wp-d': 'd2', 'wp-e': 'e2', 'wp-f': 'f2', 'wp-g': 'g2', 'wp-h': 'h2',
      'bp-a': 'a7', 'bp-b': 'b7', 'bp-c': 'c7', 'bp-d': 'd7', 'bp-e': 'e7', 'bp-f': 'f7', 'bp-g': 'g7', 'bp-h': 'h7',
      'br-L': 'a8', 'bn-L': 'b8', 'bb-L': 'c8', 'bq':   'd8', 'bk':   'e8', 'bb-R': 'f8', 'bn-R': 'g8', 'br-R': 'h8'
}
list_pieces = list(initial_chessboard.keys())

In [117]:
test_white = test[0::2]
test_black = test[1::2]

**Pawns**

In [132]:
def white_pawn(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)

    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        # if our piece is moved
        if w[0] == current_position[0] and (int(w[-1]) == int(current_position[1]) + 1 or w[-1] == '4' and current_position[1] == '2'):  
            if w[1] == 'x':
                current_position = w[2:]
            else:
                current_position = w       
        # update position_list
        positions.append(current_position)
        # if piece gets eaten
        if b[-3:] == f'x{current_position}':
            current_position = '0'
    return positions

In [133]:
def black_pawn(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)

    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        # if piece gets eaten
        if w[-3:] == f'x{current_position}':
            current_position = '0'
        # if our piece is moved
        if b[0] == current_position[0] and (int(b[-1]) == int(current_position[1]) - 1 or b[-1] == '5' and current_position[1] == '7'):  
            if b[1] == 'x':
                current_position = b[2:]
            else:
                current_position = b       
        # update position_list
        positions.append(current_position)
        
    return positions    

**Queen**

In [134]:
def white_queen(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)
    
    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        # Queen moves
        if w[0] == 'Q':
            current_position = w[-2:]
        # Queen gets eaten
        if b[-3:] == f'x{current_position}':
            current_position = '0'
        positions.append(current_position)
    return positions

In [135]:
def black_queen(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)
    
    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        # Queen gets eaten
        if w[-3:] == f'x{current_position}':
            current_position = '0'
        # Queen moves
        if b[0] == 'Q':
            current_position = b[-2:]

        positions.append(current_position)
    return positions

**King**

In [137]:
def white_king(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)
    
    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        if w == 'O-O':
            current_position = 'g1'
        if w == 'O-O-O':
            current_position = 'c1'
        if w[0] == 'K':
            current_position = w[-2:]
            
        positions.append(current_position)
    return positions

In [138]:
def black_king(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)
    
    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        if b == 'O-O':
            current_position = 'g8'
        if b == 'O-O-O':
            current_position = 'c8'
        if b[0] == 'K':
            current_position = b[-2:]
            
        positions.append(current_position)
    return positions

**Rooks**

In [ ]:
def white_rook(test_white, test_black, piece):
    current_position = initial_chessboard[piece]
    positions = []
    positions.append(current_position)
    
    for w, b in zip(test_white, test_black):
        
        w = w.replace('+', '').replace('#', '')
        b = b.replace('+', '').replace('#', '')
        
        if w == 'O-O':
            current_position = 'f1'
        if w == 'O-O-O':
            current_position = 'd1'
        if w[0] == 'R' and (w[-1] )

In [140]:
white_king(test_white, test_black, 'wk')

['e1',
 'e1',
 'e1',
 'e1',
 'e1',
 'g1',
 'g1',
 'g1',
 'g1',
 'g1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1',
 'h1']